<a href="https://colab.research.google.com/github/jiyeonjin/0624_new/blob/main/0729_PeopleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics yt-dlp opencv-python

In [ ]:
# ✅ 1. 라이브러리 설치
!pip install -q ultralytics yt-dlp opencv-python

# ✅ 2. YouTube 영상 다운로드
import yt_dlp
import os

# 유튜브 링크 입력 (여기만 바꾸면 돼!)
youtube_url = "https://youtu.be/SzRzYvQq0aQ?si=T6afHCr9eThVKcoH"

# 저장 폴더 생성
os.makedirs("videos", exist_ok=True)
input_video_path = "videos/input_video.mp4"

# yt-dlp 설정
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'outtmpl': input_video_path,
    'merge_output_format': 'mp4',
}

# 다운로드 수행
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(["https://youtu.be/SzRzYvQq0aQ?si=T6afHCr9eThVKcoH"])

print(f"✅ YouTube 영상 다운로드 완료: {input_video_path}")

# ✅ 3. YOLOv8 모델 불러오기
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # 가볍고 빠른 버전 (n), 더 정확한 건 yolov8s.pt 등 사용 가능

# ✅ 4. 영상에서 사람 인식 후 저장
import cv2

output_video_path = "videos/output_video.mp4"

cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
current_frame = 0

print("🔄 영상 처리 중...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8으로 사람(class 0) 인식
    results = model(frame, classes=[0], verbose=False)[0]

    # 바운딩 박스 표시
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        label = f"Person {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

    current_frame += 1
    if current_frame % 30 == 0:
        print(f"⏱ 처리 중: {current_frame}/{frame_count} 프레임")

cap.release()
out.release()
print(f"✅ 영상 저장 완료: {output_video_path}")

# ✅ 5. 다운로드 링크 제공
from IPython.display import FileLink, display
print("⬇️ 아래 링크를 클릭해 결과 영상을 다운로드하세요:")
display(FileLink(output_video_path))
